In [0]:
# COMMAND 1 - Verificar Catálogo e Schemas
print("=== Verificando Catálogo e Schemas ===")
display(spark.sql("SHOW CATALOGS"))
spark.sql("USE CATALOG nyc_taxi_catalog")
display(spark.sql("SHOW SCHEMAS"))

# COMMAND 2 - Verificar External Locations
print("\n=== Verificando External Locations ===")
display(spark.sql("SHOW EXTERNAL LOCATIONS"))

# COMMAND 3 - Testar Acesso aos Buckets
print("\n=== Testando Acesso aos Buckets ===")
from datetime import datetime

# Testar escrita/leitura em cada camada
layers = ['bronze', 'silver', 'gold']
for layer in layers:
    try:
        test_content = f"NYC Taxi Pipeline Test - {layer} - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        test_path = f"s3://nyc-taxi-{layer}-lucas/test.txt"
        
        dbutils.fs.put(test_path, test_content, overwrite=True)
        
        content = dbutils.fs.head(test_path)
        
        print(f"✅ {layer.upper()}: Teste de escrita/leitura OK")
        print(f"   Conteúdo: {content}")
    except Exception as e:
        print(f"❌ {layer.upper()}: Erro - {str(e)}")

# COMMAND 4 - Criar Tabelas de Teste
print("\n=== Criando Tabelas de Teste ===")
for layer in layers:
    retries = 3
    for attempt in range(retries):
        try:
            spark.sql(f"""
            CREATE TABLE IF NOT EXISTS nyc_taxi_catalog.{layer}.test (
                id INT,
                test_date TIMESTAMP,
                message STRING
            )
            LOCATION 's3://nyc-taxi-{layer}-lucas/test/'
            """)
            
            spark.sql(f"""
            INSERT INTO nyc_taxi_catalog.{layer}.test
            VALUES (1, current_timestamp(), 'Test {layer} layer - {datetime.now()}') -- Commented out to prevent long execution times
            """)
            
            print(f"✅ {layer.upper()}: Tabela de teste criada e populada")
            
            display(spark.sql(f"SELECT * FROM nyc_taxi_catalog.{layer}.test"))
            break
        except Exception as e:
            if attempt < retries - 1:
                print(f"❌ {layer.upper()}: Erro - {str(e)}. Retrying...")
            else:
                print(f"❌ {layer.upper()}: Erro - {str(e)}. Exceeded maximum retries.")

=== Verificando Catálogo e Schemas ===


catalog
main
nyc_taxi_catalog
samples
system


databaseName
bronze
default
gold
information_schema
silver



=== Verificando External Locations ===


name,url,comment
bronze_location,s3://nyc-taxi-bronze-lucas/,Bronze layer - dados raw NYC Taxi
gold_location,s3://nyc-taxi-gold-lucas/,Gold layer - dados agregados NYC Taxi
metastore_root_location,s3://nyc-taxi-managed-lucaslovato/,"Auto-created external location which provides access to the nominated metastore-level storage account for the metastore. Changing the URL on this external location will not update the metastore-level storage, and could break access. You can update the credential on this external location if desired."
silver_location,s3://nyc-taxi-silver-lucas/,Silver layer - dados limpos NYC Taxi


...

id,test_date,message
1,2025-07-30T14:29:18.719Z,Test bronze layer - 2025-07-30 14:29:17.693031
1,2025-07-30T14:30:28.399Z,Test bronze layer - 2025-07-30 14:30:28.085548


✅ SILVER: Tabela de teste criada e populada


id,test_date,message
1,2025-07-30T14:29:25.934Z,Test silver layer - 2025-07-30 14:29:25.481608
1,2025-07-30T14:30:31.245Z,Test silver layer - 2025-07-30 14:30:30.129808


✅ GOLD: Tabela de teste criada e populada


id,test_date,message
1,2025-07-30T14:29:28.911Z,Test gold layer - 2025-07-30 14:29:28.633297
1,2025-07-30T14:30:35.098Z,Test gold layer - 2025-07-30 14:30:34.762664
